In [1]:
import torch
from clustering import KNN
import pandas as pd

#load data from file
train_df=pd.read_csv("data/train.csv")
test_df=pd.read_csv("data/test.csv")

## load the embeddings from saved file
epoch4_roberta_train_emb = torch.load('out/epoch4_roberta_30379.pt', map_location="cpu")
epoch4_roberta_test_emb = torch.load('out/epoch4_roberta_17723.pt', map_location="cpu")

# gpt_train_emb = torch.load('out/gpt_30379.pt', map_location="cpu")
# gpt_test_emb = torch.load('out/gpt_17723.pt', map_location="cpu")

# bert_train_emb = torch.load('out/bert_30379.pt', map_location="cpu")
# bert_test_emb = torch.load('out/bert_17723.pt', map_location="cpu")

# codebert_train_emb = torch.load('out/codebert_30379.pt', map_location="cpu")
# codebert_test_emb = torch.load('out/codebert_17723.pt', map_location="cpu")

# roberta_train_emb = torch.load('out/roberta_30379.pt', map_location="cpu")
# roberta_test_emb = torch.load('out/roberta_17723.pt', map_location="cpu")


In [11]:
train_df["combine"]=train_df["task"]+"@"+train_df["language"]
test_df["combine"]=test_df["task"]+"@"+test_df["language"]

In [12]:
train_df.head(10)

,Unnamed: 0,task,language,code,combine
0,1,Call-an-object-method,Ada,package body My_Class is\n procedure P...,Call-an-object-method@Ada
1,2,Call-an-object-method,Ada,package Other_Class is\n type Object i...,Call-an-object-method@Ada
2,3,Call-an-object-method,Ada,package My_Class is\n type Object is t...,Call-an-object-method@Ada
3,5,Call-an-object-method,Pike,"module.func();\nmodule[""func""]();\n",Call-an-object-method@Pike
4,6,Call-an-object-method,Pike,function func = obj->method;\nfunc();\n,Call-an-object-method@Pike
5,19,Call-an-object-method,Go,procedure main()\n\n bar := foo() # create ...,Call-an-object-method@Go
6,20,Call-an-object-method,Go,"package main\n\nimport ""box""\n\nfunc main() {\...",Call-an-object-method@Go
7,21,Call-an-object-method,Go,"package box\n\nimport ""sync/atomic""\n\nvar sn ...",Call-an-object-method@Go
8,30,Call-an-object-method,LFE,(defmodule aquarium\n (export all))\n\n(defun ...,Call-an-object-method@LFE
9,31,Call-an-object-method,LFE,(defmodule object\n (export all))\n\n(defun fi...,Call-an-object-method@LFE


In [26]:
train_df=train_df.dropna()
test_df=test_df.dropna()

In [27]:
from sklearn.preprocessing import LabelEncoder

In [49]:
train_label_lang = list(train_df['combine'])
test_label_lang =list(test_df['combine']) 
all_label=train_label_lang + test_label_lang

In [50]:
print(len(train_label_lang),len(test_label_lang),len(all_label))

30365 17715 48080


In [51]:
le = LabelEncoder()
le=le.fit(all_label)


In [52]:
train_label_encoding=le.transform(train_label_lang)
test_label_encoding=le.transform(test_label_lang)

In [53]:
train_y = torch.tensor(train_label_encoding)
test_y = torch.tensor(test_label_encoding)

In [ ]:
knn_epoch4_roberta_lang = KNN(epoch4_roberta_train_emb, train_y)
knn_epoch4_roberta_task = KNN(epoch4_roberta_train_emb, train_y)

In [2]:
train_label_lang = train_df['combine']
test_label_lang = test_df['combine']
lang_dict = dict(zip(test_label_lang.unique(), [i for i in range(len(test_label_lang.unique()))]))

train_y_lang = train_label_lang.map(lang_dict)
test_y_lang = test_label_lang.map(lang_dict)
train_y_lang = torch.tensor(train_y_lang)
test_y_lang = torch.tensor(test_y_lang)

print(train_y_lang)
print(len(train_df), len(test_df), len(train_y_lang), len(test_y_lang))

tensor([ 8,  8,  8,  ..., 18, 89, 60])
30379 17723 30379 17723


In [3]:
train_label_task = train_df['task']
test_label_task = test_df['task']
task_dict = dict(zip(test_label_task.unique(), [i for i in range(len(test_label_task.unique()))]))


train_y_task = train_label_task.map(task_dict)
test_y_task = test_label_task.map(task_dict)
train_y_task = torch.tensor(train_y_task)
test_y_task = torch.tensor(test_y_task)

print(train_y_task)
print(len(train_df), len(test_df), len(train_y_task), len(test_y_task))

tensor([141, 141, 141,  ..., 741, 196, 196])
30379 17723 30379 17723


In [4]:
knn_epoch4_roberta_lang = KNN(epoch4_roberta_train_emb, train_y_lang)
knn_epoch4_roberta_task = KNN(epoch4_roberta_train_emb, train_y_task)

# knn_gpt_lang = KNN(gpt_train_emb, train_y_lang)
# knn_gpt_task = KNN(gpt_train_emb, train_y_task)

# knn_bert_lang = KNN(bert_train_emb, train_y_lang)
# knn_bert_task = KNN(bert_train_emb, train_y_task)

# knn_codebert_lang = KNN(codebert_train_emb, train_y_lang)
# knn_codebert_task = KNN(codebert_train_emb, train_y_task)

# knn_roberta_lang = KNN(roberta_train_emb, train_y_lang)
# knn_roberta_task = KNN(roberta_train_emb, train_y_task)

In [5]:
from prediction import predict

# import prediction
# import importlib 
# importlib.reload(prediction)

In [6]:
predict(epoch4_roberta_test_emb, knn_epoch4_roberta_lang, knn_epoch4_roberta_task, test_y_lang, test_y_task) 

Time elapsed: 0.15 seconds, Data predicted:0
Time elapsed: 10.39 seconds, Data predicted:1000
Time elapsed: 10.34 seconds, Data predicted:2000
Time elapsed: 10.36 seconds, Data predicted:3000
Time elapsed: 10.36 seconds, Data predicted:4000
Time elapsed: 10.34 seconds, Data predicted:5000
Time elapsed: 10.30 seconds, Data predicted:6000
Time elapsed: 10.31 seconds, Data predicted:7000
Time elapsed: 10.37 seconds, Data predicted:8000
Time elapsed: 10.36 seconds, Data predicted:9000
Time elapsed: 10.36 seconds, Data predicted:10000
Time elapsed: 10.43 seconds, Data predicted:11000
Time elapsed: 10.40 seconds, Data predicted:12000
Time elapsed: 10.33 seconds, Data predicted:13000
Time elapsed: 10.43 seconds, Data predicted:14000
Time elapsed: 10.35 seconds, Data predicted:15000
Time elapsed: 10.31 seconds, Data predicted:16000
Time elapsed: 10.57 seconds, Data predicted:17000
Accuracy of Language prediction:  1.34%
Accuracy of Language prediction:  0.11%


In [8]:
predict(gpt_test_emb, knn_gpt_lang, knn_gpt_task, test_y_lang, test_y_task) 

Time elapsed: 0.25 seconds, Data predicted:0
Time elapsed: 10.76 seconds, Data predicted:1000
Time elapsed: 10.72 seconds, Data predicted:2000
Time elapsed: 10.69 seconds, Data predicted:3000
Time elapsed: 10.69 seconds, Data predicted:4000
Time elapsed: 10.63 seconds, Data predicted:5000
Time elapsed: 10.81 seconds, Data predicted:6000
Time elapsed: 10.58 seconds, Data predicted:7000
Time elapsed: 10.68 seconds, Data predicted:8000
Time elapsed: 10.61 seconds, Data predicted:9000
Time elapsed: 10.57 seconds, Data predicted:10000
Time elapsed: 10.67 seconds, Data predicted:11000
Time elapsed: 10.66 seconds, Data predicted:12000
Time elapsed: 10.64 seconds, Data predicted:13000
Time elapsed: 10.65 seconds, Data predicted:14000
Time elapsed: 10.64 seconds, Data predicted:15000
Time elapsed: 10.61 seconds, Data predicted:16000
Time elapsed: 10.67 seconds, Data predicted:17000
Accuracy of Language prediction:  39.07%
Accuracy of Language prediction:  6.62%


In [9]:
predict(bert_test_emb, knn_bert_lang, knn_bert_task, test_y_lang, test_y_task)

Time elapsed: 0.10 seconds, Data predicted:0
Time elapsed: 10.55 seconds, Data predicted:1000
Time elapsed: 10.59 seconds, Data predicted:2000
Time elapsed: 10.67 seconds, Data predicted:3000
Time elapsed: 10.56 seconds, Data predicted:4000
Time elapsed: 11.17 seconds, Data predicted:5000
Time elapsed: 10.69 seconds, Data predicted:6000
Time elapsed: 10.97 seconds, Data predicted:7000
Time elapsed: 11.13 seconds, Data predicted:8000
Time elapsed: 11.19 seconds, Data predicted:9000
Time elapsed: 10.98 seconds, Data predicted:10000
Time elapsed: 10.91 seconds, Data predicted:11000
Time elapsed: 10.56 seconds, Data predicted:12000
Time elapsed: 10.60 seconds, Data predicted:13000
Time elapsed: 10.54 seconds, Data predicted:14000
Time elapsed: 10.57 seconds, Data predicted:15000
Time elapsed: 10.55 seconds, Data predicted:16000
Time elapsed: 12.39 seconds, Data predicted:17000
Accuracy of Language prediction:  52.19%
Accuracy of Language prediction:  16.59%


In [10]:
predict(codebert_test_emb, knn_codebert_lang, knn_codebert_task, test_y_lang, test_y_task)

Time elapsed: 0.15 seconds, Data predicted:0
Time elapsed: 11.59 seconds, Data predicted:1000
Time elapsed: 11.49 seconds, Data predicted:2000
Time elapsed: 11.57 seconds, Data predicted:3000
Time elapsed: 11.73 seconds, Data predicted:4000
Time elapsed: 11.60 seconds, Data predicted:5000
Time elapsed: 11.70 seconds, Data predicted:6000
Time elapsed: 11.75 seconds, Data predicted:7000
Time elapsed: 11.41 seconds, Data predicted:8000
Time elapsed: 11.58 seconds, Data predicted:9000
Time elapsed: 12.03 seconds, Data predicted:10000
Time elapsed: 11.46 seconds, Data predicted:11000
Time elapsed: 10.83 seconds, Data predicted:12000
Time elapsed: 10.42 seconds, Data predicted:13000
Time elapsed: 10.33 seconds, Data predicted:14000
Time elapsed: 10.31 seconds, Data predicted:15000
Time elapsed: 10.33 seconds, Data predicted:16000
Time elapsed: 10.25 seconds, Data predicted:17000
Accuracy of Language prediction:  49.26%
Accuracy of Language prediction:  10.02%


In [11]:
predict(roberta_test_emb, knn_roberta_lang, knn_roberta_task, test_y_lang, test_y_task)

Time elapsed: 0.12 seconds, Data predicted:0
Time elapsed: 10.32 seconds, Data predicted:1000
Time elapsed: 10.30 seconds, Data predicted:2000
Time elapsed: 10.51 seconds, Data predicted:3000
Time elapsed: 10.31 seconds, Data predicted:4000
Time elapsed: 10.35 seconds, Data predicted:5000
Time elapsed: 10.43 seconds, Data predicted:6000
Time elapsed: 10.23 seconds, Data predicted:7000
Time elapsed: 10.33 seconds, Data predicted:8000
Time elapsed: 10.39 seconds, Data predicted:9000
Time elapsed: 10.15 seconds, Data predicted:10000
Time elapsed: 10.26 seconds, Data predicted:11000
Time elapsed: 10.21 seconds, Data predicted:12000
Time elapsed: 10.36 seconds, Data predicted:13000
Time elapsed: 10.35 seconds, Data predicted:14000
Time elapsed: 10.33 seconds, Data predicted:15000
Time elapsed: 10.37 seconds, Data predicted:16000
Time elapsed: 10.37 seconds, Data predicted:17000
Accuracy of Language prediction:  1.34%
Accuracy of Language prediction:  0.11%


# part data on CodeBERT

In [28]:
languages = ['Go','Java','JavaScript','PHP','Python','Ruby']
indices_train = train_df.index[train_df['language'].isin(languages)].tolist()
indices_test = test_df.index[test_df['language'].isin(languages)].tolist()
print(len(indices_train), len(indices_test))

3852 1924


In [29]:
codebert_train_emb_sub = torch.index_select(codebert_train_emb, 0, torch.tensor(indices_train))
train_y_task_sub = torch.index_select(train_y_task, 0, torch.tensor(indices_train))
train_y_lang_sub = torch.index_select(train_y_lang, 0, torch.tensor(indices_train))

codebert_test_emb_sub = torch.index_select(codebert_test_emb, 0, torch.tensor(indices_test))
test_y_task_sub = torch.index_select(test_y_task, 0, torch.tensor(indices_test))
test_y_lang_sub = torch.index_select(test_y_lang, 0, torch.tensor(indices_test))

In [30]:
knn_codebert_lang_sub = KNN(codebert_train_emb_sub, train_y_lang_sub)
knn_codebert_task_sub = KNN(codebert_train_emb_sub, train_y_task_sub)

predict(codebert_test_emb_sub, knn_codebert_lang_sub, knn_codebert_task_sub, test_y_lang_sub, test_y_task_sub)

Time elapsed: 0.06 seconds, Data predicted:0
Time elapsed: 2.09 seconds, Data predicted:1000
Accuracy of Language prediction:  88.41%
Accuracy of Language prediction:  11.33%
